## Setup paquetes

In [23]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

Opcional: Montar google drive

In [24]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## Descarga y preprocesamiento de datos

Hay cambios del notebook de partida (de encoding `utf-8` a `latin-1`).

In [25]:
# texto = open("/content/gdrive/MyDrive/Colab Notebooks/quijote.txt", 'rb').read().decode(encoding='latin-1') # Para uso con google drive
texto = open("3_textos_literatura_española/quijote.txt", 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(texto)))

vocab = sorted(set(texto))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        2071308 carácteres
El texto está compuesto de estos 90 carácteres:
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Í', 'Ó', 'Ú', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü']


### Procesamiento de los textos
#### Mapeo de caracteres

In [26]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '0' :  10,
  '1' :  11,
  '2' :  12,
  '3' :  13,
  '4' :  14,
  '5' :  15,
  '6' :  16,
  '7' :  17,
  ':' :  18,
  ';' :  19,
  '?' :  20,
  'A' :  21,
  'B' :  22,
  'C' :  23,
  'D' :  24,
  'E' :  25,
  'F' :  26,
  'G' :  27,
  'H' :  28,
  'I' :  29,
  'J' :  30,
  'L' :  31,
  'M' :  32,
  'N' :  33,
  'O' :  34,
  'P' :  35,
  'Q' :  36,
  'R' :  37,
  'S' :  38,
  'T' :  39,
  'U' :  40,
  'V' :  41,
  'W' :  42,
  'X' :  43,
  'Y' :  44,
  'Z' :  45,
  ']' :  46,
  'a' :  47,
  'b' :  48,
  'c' :  49,
  'd' :  50,
  'e' :  51,
  'f' :  52,
  'g' :  53,
  'h' :  54,
  'i' :  55,
  'j' :  56,
  'l' :  57,
  'm' :  58,
  'n' :  59,
  'o' :  60,
  'p' :  61,
  'q' :  62,
  'r' :  63,
  's' :  64,
  't' :  65,
  'u' :  66,
  'v' :  67,
  'x' :  68,
  'y' :  69,
  'z' :  70,
  '¡' :  71,
  '«' :  72,
  '»' :  73,
  '¿' :  74,
  'Á' :  75,
  'É' :  76,

Pasamos cada texto a un array de enteros

In [27]:
text_as_int = np.array([char2idx[c] for c in texto])

print ('texto : {}'.format(repr(texto[:50])))
print ('{}'.format(repr(texto[:50])))

texto : 'El ingenioso hidalgo don Quijote de la Mancha\n\n\nPr'
'El ingenioso hidalgo don Quijote de la Mancha\n\n\nPr'


### Preparación de los datos para entrenar la RNN

Para entrenar el modelo creamos un conjunto de datos con el contenido de text_as_init. Para ello utilizamos la función tf.data.Dataset.from_tensor_slices.
A este conjunto de datos lo dividiremos en secuencias de seq_length+1 al aplicar el método batch()


In [28]:
# Creamos una función `split_input_target` que devolverá el conjunto de datos
# de entrenamiento (los datos de entrada como los datos de salida)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#Agrupamos los dataset en batches de 64 .
# Así tendriamos los datos de entrenamiento con batches compuestos de 64 parejas
# de secuencias de 100 integers de 64 bits
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [29]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

#Aplicamos split_input_target a todas las secuencias utilizando el método map()
dataset = sequences.map(split_input_target)

print("\n\n")

#Los dataset contienen un conjunto de parejas (100 caracteres del texto original, la correspondiente salida ). Vamos a mostrar la primera pareja.
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

  print(dataset)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

'El ingenioso hidalgo don Quijote de la Mancha\n\n\nPrimera parte del ingenioso hidalgo don Quijote de la'
' Mancha\n\nCapítulo primero. Que trata de la condición y ejercicio del famoso hidalgo\ndon Quijote de la'
' Mancha\n\n\nEn un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho\ntiempo que vivía '
'un hidalgo de los de lanza en astillero, adarga antigua,\nrocín flaco y galgo corredor. Una olla de al'
'go más vaca que carnero,\nsalpicón las más noches, duelos y quebrantos los sábados, lantejas los\nviern'
'es, algún palomino de añadidura los domingos, consumían las tres\npartes de su hacienda. El resto dell'
'a concluían sayo de velarte, calzas de\nvelludo para las fiestas, con sus pantuflos de lo mesmo, y los'
' días de\nentresemana se honraba con su vellorí de lo más fino. Tenía en su casa una\nama que pasaba de'
' los cuarenta, y una sobrina que no llegaba a los veinte,\ny un mozo de campo y plaza, que así ensilla'
'ba el rocín como tomaba la\npodadera. Frisaba

### Construcción del modelo RNN

In [30]:
#Crearemos una función que cree un modelo RNN con tres capas
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  #Añadimos la capa de tipo word embedding
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      #batch_input_shape=[batch_size, None] Deprecated
                      ))
  #Añadimos la capa de tipo LSTM
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(512, activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))


  #Añadimos la capa de tipo Dense
  model.add(Dense(vocab_size))
  return model

In [31]:
embedding_dim = 256
rnn_units = 1024

In [32]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print("Prediction : ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction :  (64, 100, 90) # (batch_size, sequence_length, vocab_size)


2025-06-24 22:35:24.664422: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [36]:
print(sampled_indices_characters)

[ 8 64 73 63 10 71  0 28 16 65 16 63  6 33 39 19 48  7 58  4 84 81 67  7
 64  6 26 39 70  0 30  6  7 58 18 85 29 65 66 32 52 54 18 60 84 87 48 34
 69 14 17 59 10 70 71 74 75 82 67 43 69 11 84 30 66 68 85  4 33 64 13 88
 17 83 73 18  0 19 17 20 18 22 71 86 16 15  6 10 80 56 57 78 24 70 22 60
 57 50  1  3]


### Entrenamiento del modelo RNN

In [37]:
#Creamos la función de perdida, usaremos el categorical pues estamos considerando datos categóricos
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [38]:
#Compilamos el modelo
model.compile(optimizer='adam', loss=loss)

In [39]:
#configuramos los checkpoints

checkpoint_dir = './training_checkpoints_Quijote'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [40]:
# Implementación EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(
    monitor='loss',
    patience=10,
    min_delta=0.01,
    restore_best_weights=True
)

In [41]:
#Entrenamos el modelo
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, early_stopping_callback])

Epoch 1/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 190s 592ms/step - loss: 2.3630
Epoch 2/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 199s 621ms/step - loss: 1.6249
Epoch 3/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 205s 639ms/step - loss: 1.4342
Epoch 4/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 212s 661ms/step - loss: 1.3381
Epoch 5/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 211s 657ms/step - loss: 1.2780
Epoch 6/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 205s 639ms/step - loss: 1.2303
Epoch 7/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 206s 641ms/step - loss: 1.1921
Epoch 8/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 206s 643ms/step - loss: 1.1577
Epoch 9/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 206s 643ms/step - loss: 1.1277
Epoch 10/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 208s 649ms/step - loss: 1.0988
Epoch 11/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 208s 649ms/step - loss: 1.0711
Epoch 12/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 209s 651ms/step - loss: 1.0421
Epoch 13/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 212s 661ms/step - loss: 1.0143
Epoch 14/50
320/320 ━━━━━━━━━━━━━━━━━━━━ 216s 675ms/step - loss: 0.9897
E

In [42]:
model.save("model_quijote_100_2025.keras")

In [43]:
from keras.models import load_model
from keras import losses # Import the losses module

# Assuming your original loss function was, for example, binary_crossentropy
loaded_model = load_model("model_quijote_100_2025.keras",
                          custom_objects={'loss': losses.sparse_categorical_crossentropy})
# or if it was a custom loss function
# loaded_model = load_model("model_paquita_100_2024.keras", custom_objects={'loss': my_custom_loss_function})

In [44]:
model = build_model(len(vocab), embedding_dim, rnn_units, batch_size=1)
input_shape = (1, 100)  # Replace 100 with your actual sequence length
model.build(input_shape=input_shape) # Or model.build(tf.TensorShape([1, None]))

model.load_weights("model_quijote_100_2025.keras")

In [49]:
#Creamos una función generar_texto que generará texto a partir de una palabra de partida
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  # temperature = 0.7 # cuanto mas alto el numero, mas directa la salida de los logits
  temperature = 0.2 # con 0.3 va mejor segun testeo con la otra clase, con 0.1 se vuelve repetitivo

#  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [50]:
print(generate_text(model, start_string=u"En un lugar de la Mancha, de cuyo nombre no quiero acordarme"))

En un lugar de la Mancha, de cuyo nombre no quiero acordarme en
volver tan al cielo abierto, pues tan saltres como el de
Cardenio, que ya no estoy cansado de mentir todo mi voluntad y tal
medro. Y, siendo yo la maldiciendo en que ella le había dado. Viéndole así
dar a los dos que han de servir cuatro cocos de a caballo y de los atrevidos que la
mercedes se hallan. Yo no he visto ni otra cosa de más de
cuatro días, yo me estimo yo en mi casa, y por esto como por mi provecho. Yo, Sancho,
bien criado, corazón de acompañar a la soledad de los nombres: a tiempo que la había de llevar al
lugar de don Quijote, y así lo confirmó a buscar a don Quijote, por parecerle que se le
desengañaba, de modo que la abriente la sala por los caminos podía
ver más a menudo y todos los demás simplicidades que se debía a
tiempo que la valentía y el de su señor había escuchado. Y, al
querer verdad a lo que se le ha de hacer dellos, como vuestra merced lleva el conveniente rico, entre
otras, muchos son las que a

# Resultados, análisis y conclusiones

## Intento 1: 
**50 Épocas y 0.3 de temperatura**
```
En un lugar de la Mancha, de cuyo nombre no quiero acordarme en ello, porque
tenga fama de tan buena suerte, y amiga mía, a quien tiene lleno de memorias y
las que le dieron con sus armas y caballo que se hallaba en Argel, edvierta sangre de la malicia del alma.
   Tú te guardas, y que esta mañana me ha dado y
escribiendo en la tabla del mundo. Y, porque no pierdas, Sancho me daré de que todos los de tu atrevimiento y
su corazón hace, no se os ha caballero, y de los mejores muestran todos los días de mi vida?

-Sí diré, pecador de mí, -replicó Sancho-, porque los buenos decir que eres, ya en este
paracer, de hombre de caña, como mostraba el arroyo en mi espada, con la lanza son religiosas más estrechas aguas, y
después me desembaraza, y así lo temió y dijo:

-Señor mío, yo no debo de entender que no soy de dos libros menos simples que las han
vengado, no nos descontara; pero, en fin, se halló en unas florestas trabajos y
volverme a pedor de comer; y, como no la han dicho que de mí se debía de haber
hecho alguna noche. Dulcinea es mi esposa, y l
```
### Observaciones
Está bastante bien para un modelo entrenado poco. El tono es fiel a la obra original, mezcla frases que suenan naturales con otras que se le van un poco de las manos. Algunas partes son fluidas y se entiende lo que intenta decir, pero no llega a ser coherente. No repite tanto directo del original, pero todavía le cuesta mantener una idea.

## Intento 2:
**50 Épocas y 0.4 de temperatura**
```
En un lugar de la Mancha, de cuyo nombre no quiero acordarme de
nodados bien las cosas.

-Eso creo yo muy bien -respondió el bachille-, que no quiero llevar el licenciado de mis dueños, y
a pesar sus labios, y que en el traje parece que con las virtudes tienen más
de los que me conocen y conocen su mujer, sin que ha de tomar al que de llamarte es el
que tengo de servirse, y comer alguna rodela al mayor concebido en los
principios, con el pelo de la venta, que a él le pareció que
todos los demás saltando con sus amigos sus padres, así como estaba, y al cual no le contentará lo que debo
a mi señora, aunque no me lo pagare a ponerme en ella, y de algún grave acocida a quien esta noche
ha he dicho, no se me ha de dar nada para que se los deje
de decir otras noches las que aquí vienen que los reyes y profesión, que de la honestidad son de mi vida, a lo
menos, estaré yo más escondida de mi muerte.

-Desa manera -respondió don Quijote-, porque estos dos amigos me lo
dije y acarrea de los reyes como las humildes chozas de la cabeza, y la
doncella sube
```

### Observaciones
Esta versión del modelo tiene un tono más creativo (temperatura más alta) y algunas frases que no tienen mucho sentido pero suenan del estilo del Quijote. Hay saltos temáticos abruptos y algunas partes no tienen ningún sentido.